### Library and model imports (execute just once)

In [8]:
from utils import list_images, generate_multichannel_tif, remap_labels, remove_border_cells, extract_features, extract_img_metadata
from tqdm import tqdm
from cellpose import models, core
from tifffile import imread
from pathlib import Path
import numpy as np
import os

# Ignore OME-TIFF warnings during imread
import logging
logging.getLogger("tifffile").setLevel(logging.ERROR)

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM
model = models.CellposeModel(gpu=True)

### Define the path containing your images

In [9]:
# Copy the path to the folder containing your images in between the quotation marks
data_folder = r"Z:\MariiaY\Evos Anna nanoparticles\Ex7_neg_LNPs_.2025-11-27-11-47-57"

# If the path is correct you should see a list of the first 10 images in your folder down below
images = list_images(data_folder, format="tif")
images[:10]

['Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f00d1.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f00d2.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f00d4.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f01d1.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f01d2.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f01d4.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f02d1.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.2025-11-27-11-47-57\\Ex7_neg_LNPs__Plate_D_p00_0_B01f02d2.TIF',
 'Z:\\MariiaY\\Evos Anna nanoparticles\\Ex7_neg_LNPs_.20

### Preprocess single channel into multichannel 

In [10]:
# Process single channel .tif files into multichannel .tif files and save them
generate_multichannel_tif(data_folder)


Processing input single channel .tif into multichannel .tif files 

Processed .tif files will be saved under: Z:\MariiaY\Evos Anna nanoparticles\Ex7_neg_LNPs_.2025-11-27-11-47-57\processed_tiffs



100%|██████████| 560/560 [00:01<00:00, 452.39it/s]


### Batch analyze your images

In [ ]:
# Point to the folder containing the processed tiffs (leave by default)
processed_img_path = Path(data_folder) / "processed_tiffs"
processed_images = list_images(processed_img_path, format="tif")

# Extract experiment_id from data folder Path object
experiment_id = Path(data_folder).name

# Create a 'results' folder in the root directory
results_folder = Path("results") / experiment_id 

try:
    os.makedirs(results_folder)
    print(f"'{results_folder}' folder created successfully.")
except FileExistsError:
    print(f"'{results_folder}' folder already exists.")

# Define the .csv name
csv_name = "avg_int_per_cell_data.csv"

# Check if there is a previous copy of the .csv file at results_folder / csv_name 
csv_path = results_folder / csv_name
# Delete it to avoid potential issues
if csv_path.is_file():
    print(f"Removing existing file before recalculating results: {csv_path}")
    try:
        csv_path.unlink()
    except FileNotFoundError:
        pass

for img_filepath in tqdm(processed_images):

    # Read .tif file
    img = imread(img_filepath)

    # Extract image metadata from filename
    descriptor_dict = extract_img_metadata (img_filepath, verbose = False)

    # Predict nuclei labels using CellposeSAM 
    nuclei_labels, flows, styles = model.eval(img[1], niter=1000) # need to check the arguments

    # Predict cytoplasm labels using CellposeSAM using nuclei (1) and brightfield (2) images as input 
    cytoplasm_labels, flows, styles = model.eval((img[[1,2]]), niter=1000) # need to check the arguments

    # Label-to-label remapping: each nucleus inherits the cytoplasm label value it lies in
    nuclei_remapped = remap_labels(nuclei_labels, cytoplasm_labels)

    # Remove cell entities touching the image border
    cytoplasm_labels, nuclei_remapped = remove_border_cells(cytoplasm_labels, nuclei_remapped) 

    # Simple feature extraction (edit scikit_props if you need more features)
    props_df = extract_features(img, nuclei_remapped, cytoplasm_labels, descriptor_dict, scikit_props=["label", "intensity_mean", "area"])

    # Append to the .csv with new data points each round
    props_df.to_csv((csv_path), mode="a", index=False, header=not os.path.isfile(csv_path))

'results\Ex7_neg_LNPs_.2025-11-27-11-47-57' folder created successfully.


100%|██████████| 560/560 [32:42<00:00,  3.51s/it]
